## Montefiore SQL Test

## Using pyton to run SQL query

##### Jacobo Simkin

##### Thank you for taking the time to go over my work. It was a pleasure to complete this task.

In [4]:
# Importing library needed.

In [6]:
import pandas as pd

In [8]:
from sqlalchemy import create_engine, Table, Column, MetaData

In [10]:
# Code to actiavte a temporary SQLite database.

In [12]:
engine = create_engine('sqlite:///:memory:')

In [14]:
# File path to open it in the enviroment.

In [16]:
# URL of the raw file on GitHub
url = 'https://raw.githubusercontent.com/jacosimkin/Test/main/SQL%20Tables%20for%20DATA%20ANALYST.xlsx'

In [18]:
# Read the multple sheets on the file

In [20]:
# Read the Excel file directly from the URL
df_dept = pd.read_excel(url, sheet_name='DEPT')
df_doc_dept = pd.read_excel(url, sheet_name='DOC DEPT')
df_pat = pd.read_excel(url, sheet_name='Pat')
df_provider = pd.read_excel(url, sheet_name='Provider')
df_visit = pd.read_excel(url, sheet_name='Visit')

In [21]:
# Verifying that the DOB column is correctly formatted as datetime64

In [24]:
print("Columns and Data Types of df_pat:")
print(df_pat.dtypes)

Columns and Data Types of df_pat:
PAT_ID                object
PATIENT               object
DOB           datetime64[ns]
CUR_DOC_ID             int64
dtype: object


In [26]:
# Verifying that the VISIT_DATE column is correctly formatted as datetime

In [28]:
print("Columns and Data Types of df_visit:")
print(df_visit.dtypes)


Columns and Data Types of df_visit:
PAT_ID          object
VISIT_ID        object
DEPT_ID         object
VISIT_DOC_ID     int64
VISIT_DATE      object
dtype: object


In [30]:
# Converting VISIT_DATE into datetime

In [32]:
df_visit['VISIT_DATE'] = pd.to_datetime(df_visit['VISIT_DATE'])

In [34]:
# Code to store pandas DataFrames into SQLite database tables using SQLAlchemy.

In [36]:
df_pat.to_sql('df_pat', engine, index=False)
df_visit.to_sql('df_visit', engine, index=False)
df_provider.to_sql('df_provider', engine, index=False)
df_dept.to_sql('df_dept', engine, index=False)
df_doc_dept.to_sql('df_doc_dept', engine, index=False)


1428

In [38]:
# SQL query.

In [40]:
sql_query = """
SELECT
    P.PAT_ID,
    P.PATIENT,
    P.DOB,
    PD_CUR.DOC_NAME AS CUR_DOC,
    V.VISIT_DATE,
    V.VISIT_ID,
    PD_VISIT.DOC_NAME AS VISIT_DOC,
    D.DEPT_NAME,
    CASE WHEN D.FPG_DEPT_FLAG = 1 THEN 'Y' ELSE 'N' END AS VISIT_AT_FPG,
    CASE WHEN DD.FPG_DEPT_FLAG = 1 THEN 'Y' ELSE 'N' END AS CUR_DOC_FPG,
    CASE WHEN P.CUR_DOC_ID = V.VISIT_DOC_ID THEN 'Y' ELSE 'N' END AS CUR_DOC_IS_VIS_DOC,
    CAST((julianday(V.VISIT_DATE) - julianday(P.DOB)) / 365.25 AS INTEGER) AS Age_at_Visit
FROM
    df_pat P
JOIN
    df_visit V ON P.PAT_ID = V.PAT_ID
JOIN
    df_provider PD_CUR ON P.CUR_DOC_ID = PD_CUR.DOC_ID
JOIN
    df_provider PD_VISIT ON V.VISIT_DOC_ID = PD_VISIT.DOC_ID
JOIN
    df_dept D ON V.DEPT_ID = D.DEPT_ID
JOIN
    df_doc_dept DD ON P.CUR_DOC_ID = DD.DOC_ID AND DD.LINE = 1
WHERE
    D.FPG_DEPT_FLAG = 1;
"""

In [42]:
# Pandas is used to execute a SQL query on a database via an SQLAlchemy Engine object (engine) and store the result in a Dataframe. 

In [44]:
df_output = pd.read_sql_query(sql_query, engine)

In [46]:
# Output

In [48]:
df_output

,PAT_ID,PATIENT,DOB,CUR_DOC,VISIT_DATE,VISIT_ID,VISIT_DOC,DEPT_NAME,VISIT_AT_FPG,CUR_DOC_FPG,CUR_DOC_IS_VIS_DOC,Age_at_Visit
0,Z37470,PAT CEN,1969-01-22 00:00:00.000000,HOCDOC217,2020-05-21 00:00:00.000000,VIS-41852,QUIDOC799,EAS DEP23,Y,Y,N,51
